In [1]:
import json
import pandas as pd
import scispacy
import spacy
from tqdm import tqdm_notebook,tqdm
import random
import numpy as np

seed = 42
np.random.seed(seed)
random.seed(seed)

add sentence granularity text to the original paragraph granularity training dataset. 

load data

ony the train data


In [3]:
train_data_abs = json.load(open("../data/train.json","r"))
mechanism_data = json.load(open("../../mechan_ent_rel_ext/ann_result/ner_0730.json","r"))
mechanism_paper_dict = {}
for paper in mechanism_data:
    mechanism_paper_dict[paper["paper_id"]] = paper


use scispacy split sentences

In [4]:
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe('sentencizer')
def split_para_to_sents(text):
    doc = nlp(text)
    sents = {}
    for sent_idx , sent in enumerate(doc.sents):
        sents[sent_idx] = sent.text
    return sents

gen data

if the sentence contained an effect entity, it was labeled 1.

In [6]:
from itertools import combinations
def gen_sents_with_label_data_from_mechanism_paper(paper_id):
    paper = mechanism_paper_dict[paper_id]
    data = []
    for sent_idx,sent in paper["sents"].items():
        sent_idx = int(sent_idx)
        item = {
            "text":sent["text"],
            "sent_idx":sent_idx,
            "label":0,
            "in_mechanism_paper":1,
            "paper_id":paper["paper_id"]
        }

        if sent_idx in paper["ent_sent_idx_list"]["Effect"]:
            item["label"] = 1
            data.append(
                item
            )
        
    neg_sents = [sent["text"] for sent_idx,sent in paper["sents"].items() if int(sent_idx) not in paper["ent_sent_idx_list"]["Effect"]]
    data.append(
        {
            "text":" ".join(neg_sents).strip(),
            "sent_idx":0,
            "label":0,
            "in_mechanism_paper":1,
            "id":paper["paper_id"]
        }
    )


    pos_sents = [sent["text"] for sent_idx,sent in paper["sents"].items() if int(sent_idx) in paper["ent_sent_idx_list"]["Effect"] ]

    if len( paper["ent_sent_idx_list"]["Effect"]) > 1 :
        data.append(
        {
            "text":" ".join(pos_sents).strip(),
            "sent_idx":0,
            "label":1,
            "in_mechanism_paper":1,
            "id":paper["paper_id"]
        }
        )
    return data



In [7]:
def gen_sents_with_label_data_from_non_mechanism_paper(paper):
    data = []

    for sent_idx,sent in split_para_to_sents(paper["text"]).items():
        sent = sent.strip()
        if len(sent) < 15:
            continue
        item = {
            "text":sent,
            "sent_idx":sent_idx,
            "label":0,
            "in_mechanism_paper":0,
            "paper_id":paper["id"]
        }
        if sent_idx >= 4:
            data.append(item)
    return data

### 数据生成

In [8]:
data = []
for paper in tqdm(train_data_abs):
    if paper["id"] in mechanism_paper_dict.keys():
        temp = gen_sents_with_label_data_from_mechanism_paper( paper["id"])
        data.extend(temp)
    # else:
    #     data.extend(gen_sents_with_label_data_from_non_mechanism_paper(paper))
len(data)

100%|██████████| 1000/1000 [00:00<00:00, 167277.02it/s]


431

save data

In [19]:
random.shuffle(data)
len(data)

1431

In [ ]:
target_dir = "../data"
def save_data(saved_data,saved_name):
    json.dump(
        saved_data,open(target_dir+"/"+saved_name+".json","w",encoding="utf-8"),
        indent=2,
        ensure_ascii=False
    )
    print("save data to ",saved_name)
save_data(data,"train_with_sent_level_data")